## airtable_tools_v2

In [64]:
import os, logging
import numpy as np
from tqdm import tqdm
import pandas as pd
from pyairtable import Table
from dotenv import load_dotenv
from datetime import datetime
import pytz

In [65]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
                    datefmt='%m-%d %H:%M:%S',level='INFO')
logger=logging.getLogger(__name__)
logger.setLevel('INFO')

In [66]:
load_dotenv()
api_key=os.getenv('AIRTABLE_API_KEY')
base_id='appDTdyxcShkSR3oF'

## Logs

In [67]:
dst_table_name='[A]Logs'

In [68]:
logger.info('Connecting to Log and Habits table...')
log_table=Table(api_key, base_id, 'Log')
habit_table=Table(api_key, base_id, 'Habits')

logger.info('Connecting to %s table...', dst_table_name)
a=Table(api_key, base_id, dst_table_name)

[05-15 19:25:53] p1018 {<ipython-input-68-0324345d546c>:1} INFO - Connecting to Log and Habits table...
[05-15 19:25:53] p1018 {<ipython-input-68-0324345d546c>:5} INFO - Connecting to [A]Logs table...


In [87]:
logger.info('Finding Log records with "Log" in the Habit name...')
res=[]
for o in tqdm(log_table.all()):
    if 'fields' not in o: continue
    if 'Habit' not in o['fields']: continue
    if len(o['fields']['Habit'])>0:
        habit=habit_table.get(o['fields']['Habit'][0])
        if 'Log' in habit['fields']['Habit']:
            res.append({'Date':o['fields']['Date'], 'Score':o['fields']['Score'], 'Log': habit['fields']['Habit'].strip('Log: ')})

logger.info('Converting to dataframe and resampling...')
df=pd.DataFrame(res)
df.index=pd.DatetimeIndex(df['Date'])

logger.info('Deleting existing %s data..', dst_table_name)
r=a.all()
ids=[o['id'] for o in r]
res=a.batch_delete(ids)

logger.info('Uploading new analytics data...')
for l in list(df['Log'].unique()):
    dfs=df[df['Log']==l]
    dfs=dfs.sort_index()
    dfr=dfs.resample('D').mean() #Resample
    
    res=[]
    for d in dfr.index:
        if np.isnan(dfr.loc[d]['Score']): continue
        res.append({'Date': str(d.date()),
                 'Score': int(dfr.loc[d]['Score']),
                 'Metric': l})
    res=a.batch_create(res)
logger.info('Completed uploading %i records. ', len(res))


[05-15 19:40:50] p1018 {<ipython-input-87-afc5769bc7f9>:1} INFO - Finding Log records with "Log" in the Habit name...
100%|██████████| 100/100 [00:03<00:00, 26.34it/s]
[05-15 19:40:54] p1018 {<ipython-input-87-afc5769bc7f9>:11} INFO - Converting to dataframe and resampling...
[05-15 19:40:54] p1018 {<ipython-input-87-afc5769bc7f9>:15} INFO - Deleting existing [A]Logs data..
[05-15 19:40:55] p1018 {<ipython-input-87-afc5769bc7f9>:20} INFO - Uploading new analytics data...
[05-15 19:40:56] p1018 {<ipython-input-87-afc5769bc7f9>:33} INFO - Completed uploading 9 records. 


## Health Habits

In [100]:
dst_table_name='[A]HealthHabits'
non_weight_volume_mult=50

In [91]:
logger.info('Connecting to Log, Habits, and Goal table...')
log_table=Table(api_key, base_id, 'Log')
habit_table=Table(api_key, base_id, 'Habits')
goal_table=Table(api_key, base_id, 'Goals')

logger.info('Connecting to %s table...', dst_table_name)
a=Table(api_key, base_id, dst_table_name)

[05-15 20:14:19] p1018 {<ipython-input-91-3c083b31d03a>:1} INFO - Connecting to Log, Habits, and Goal table...
[05-15 20:14:19] p1018 {<ipython-input-91-3c083b31d03a>:7} INFO - Connecting to [A]HealthHabits table...


In [113]:
logger.info('Finding Exercise Logs')
res=[]
for o in tqdm(log_table.all()):
    if 'fields' not in o: continue
    if 'Habit' not in o['fields']: continue
    if len(o['fields']['Goal 2'])>0:
        goal=goal_table.get(o['fields']['Goal 2'][0])
        habit=habit_table.get(o['fields']['Habit'][0])['fields']['Habit']
        if 'fields' not in goal: continue
        if 'Goal' not in goal['fields']: continue
        if goal['fields']['Goal']=="Health":
            if 'Reps' in o['fields'] and 'Weight' in o['fields'] and 'Sets' in o['fields']:
                volume=float(o['fields']['Reps'])*float(o['fields']['Sets'])*float(o['fields']['Weight'])
            elif 'Minutes' in o['fields']: 
                volume=non_weight_volume_mult*o['fields']['Minutes']
            else:
                volume=100
                
            if len(o['fields']['Category'])>0: category=o['fields']['Category'][0]
            else: category='Unknown'
            res.append({'Date':o['fields']['Date'], 
                        'Volume':volume, 
                        'Habit':habit,
                        'Category': category})
            
logger.info('Converting to dataframe and resampling...')
df=pd.DataFrame(res)
df.index=pd.DatetimeIndex(df['Date'])

logger.info('Deleting existing %s data..', dst_table_name)
r=a.all()
ids=[o['id'] for o in r]
res=a.batch_delete(ids)

[05-15 20:27:42] p1018 {<ipython-input-113-f442c07a5298>:1} INFO - Finding Exercise Logs
100%|██████████| 100/100 [00:07<00:00, 13.74it/s]
[05-15 20:27:50] p1018 {<ipython-input-113-f442c07a5298>:26} INFO - Converting to dataframe and resampling...
[05-15 20:27:50] p1018 {<ipython-input-113-f442c07a5298>:30} INFO - Deleting existing [A]HealthHabits data..


In [114]:
df.head()

,Date,Volume,Habit,Category
Date,,,,
2022-05-09,2022-05-09,500.0,Peloton,Cardio
2022-05-13,2022-05-13,250.0,Peloton,Cardio
2022-05-11,2022-05-11,100.0,Chest Fly,Chest
2022-05-09,2022-05-09,100.0,Dip,Chest
2022-05-09,2022-05-09,100.0,Squat barbell,Legs


In [117]:
df=df.sort_index()
dfr=df.resample('D').sum() #Resample

In [124]:
# dfr.loc[pd.datetime.strptime('2021-4-20', '%Y-%m-%d')]=0 #Working test case
end_date=pd.datetime.now()
if end_date not in dfr.index: #Do we have data from today?
    dfr.loc[end_date]=0 #Add 0s at todays date
    dfr=dfr.resample('D').sum()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


In [125]:
logger.info('Computing cumulative sums by type...')

for s in df['Category'].unique(): dfr['Volume_'+s]=df[df['Category']==s]['Volume'].resample('D').sum()
dfr=dfr.replace(np.NaN, 0)
for s in df['Category'].unique(): dfr['Volume_Cumulative_'+s]=dfr['Volume_'+s].cumsum()

[05-15 20:30:57] p1018 {<ipython-input-125-a68a068b0f76>:1} INFO - Computing cumulative sums by type...


In [126]:
dfr

,Volume,Volume_Cardio,Volume_Chest,Volume_Legs,Volume_Arms,Volume_Back,Volume_Abs,Volume_Shoulders,Volume_Cumulative_Cardio,Volume_Cumulative_Chest,Volume_Cumulative_Legs,Volume_Cumulative_Arms,Volume_Cumulative_Back,Volume_Cumulative_Abs,Volume_Cumulative_Shoulders
Date,,,,,,,,,,,,,,,
2022-05-09,6380.0,500.0,2380.0,2050.0,1350.0,100.0,0.0,0.0,500.0,2380.0,2050.0,1350.0,100.0,0.0,0.0
2022-05-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,500.0,2380.0,2050.0,1350.0,100.0,0.0,0.0
2022-05-11,7525.0,500.0,100.0,2865.0,960.0,1440.0,100.0,1560.0,1000.0,2480.0,4915.0,2310.0,1540.0,100.0,1560.0
2022-05-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1000.0,2480.0,4915.0,2310.0,1540.0,100.0,1560.0
2022-05-13,5870.0,250.0,1920.0,3120.0,0.0,280.0,0.0,300.0,1250.0,4400.0,8035.0,2310.0,1820.0,100.0,1860.0
2022-05-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1250.0,4400.0,8035.0,2310.0,1820.0,100.0,1860.0
2022-05-15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1250.0,4400.0,8035.0,2310.0,1820.0,100.0,1860.0


In [128]:
logger.info('Deleting old Analytics data..')
r=a.all()
ids=[o['id'] for o in r]
res=a.batch_delete(ids)

[05-15 20:31:35] p1018 {<ipython-input-128-214e3df9818d>:1} INFO - Deleting old Analytics data..


In [132]:
logger.info('Uploading new analytics data...')
l=[]
for s in df['Category'].unique():
    for d in dfr.index:
        l.append({'Date': str(d.date()),
                 'Exercise Volume': dfr.loc[d]['Volume_Cumulative_'+s],
                 'Category': s})
res=a.batch_create(l)

[05-15 20:33:59] p1018 {<ipython-input-132-6841a70e9f65>:1} INFO - Uploading new analytics data...


In [129]:
# logger.info('Uploading new health data...')
# for l in list(df['Log'].unique()):
#     dfs=df[df['Log']==l]
#     dfs=dfs.sort_index()
#     dfr=dfs.resample('D').mean() #Resample
#     res=[]
#     for d in dfr.index:
#         if np.isnan(dfr.loc[d]['Score']): continue
#         res.append({'Date': str(d.date()),
#                  'Score': int(dfr.loc[d]['Score']),
#                  'Metric': l})
#     res=a.batch_create(res)
# logger.info('Completed uploading %i records. ', len(res))

In [84]:
res

[{'Date': '2022-05-07', 'Score': 5, 'Metric': 'Satisfaction'},
 {'Date': '2022-05-08', 'Score': 5, 'Metric': 'Satisfaction'},
 {'Date': '2022-05-09', 'Score': 4, 'Metric': 'Satisfaction'},
 {'Date': '2022-05-10', 'Score': 4, 'Metric': 'Satisfaction'},
 {'Date': '2022-05-11', 'Score': 4, 'Metric': 'Satisfaction'},
 {'Date': '2022-05-12', 'Score': 4, 'Metric': 'Satisfaction'},
 {'Date': '2022-05-13', 'Score': 2, 'Metric': 'Satisfaction'},
 {'Date': '2022-05-14', 'Score': 1, 'Metric': 'Satisfaction'}]

```
{
  "$schema": "https://vega.github.io/schema/vega-lite/v4.json",
  "title": "[A]Logs",
  "width": "container",
  "height": "container",
  "mark": "area",
  "encoding": {
    "x": {
      "field": "Date",
      "type": "nominal"
    },
    "y": {
      "aggregate": "sum",
      "field": "Score",
      "type": "quantitative"
    },
    "color": {
      "type":"nominal",
      "field": "Metric",
      "scale": {"scheme": "category10"}
    }
  }
}
```

In [ ]:
# logger.info('Connecting to %s table...', src_table_name)
# a=Airtable(base_id, src_table_name, api_key)

# res=a.get_all()
# logger.info('Found %i records', len(res))
# res=[{**r['fields'], **r} for r in res]

# logger.info('Converting to dataframe and resampling...')
# df=pd.DataFrame(res)
# df.index=pd.DatetimeIndex(df['Date'])
# df=df.sort_index()
# dfr=df.resample('D').mean() #Resample

# logger.info('Connecting to %s table...', dst_table_name)
# a=Airtable(base_id, dst_table_name, api_key)

# logger.info('Deleting existing %s data..', dst_table_name)
# r=a.get_all()
# ids=[o['id'] for o in r]
# res=a.batch_delete(ids)

# logger.info('Uploading new analytics data...')
# l=[]
# for s in measures:
#     for d in dfr.index:
#         if np.isnan(dfr.loc[d][s]): continue
#         l.append({'Date': str(d.date()),
#                  'Measure': dfr.loc[d][s],
#                  'Metric': s})
# res=a.batch_insert(l)
# logger.info('Completed uploading %i records. ', len(res))

---

In [43]:
res=action_table.all()
logger.info('Found %i records', len(res))
res=[{**r['fields'], **r} for r in res]

logger.info('Converting to dataframe and resampling...')
df=pd.DataFrame(res)

[05-08 20:59:44] p1144 {<ipython-input-43-f504cc177592>:2} INFO - Found 4 records
[05-08 20:59:44] p1144 {<ipython-input-43-f504cc177592>:5} INFO - Converting to dataframe and resampling...


In [44]:
df

,Date,Habit,Score,Goal 2,id,createdTime,fields
0,2022-05-07,[recFqsNfUNIe88oRK],5,[recenm1UYUwAIcoya],recMHleWhG91jn7sc,2022-05-08T19:00:41.000Z,"{'Date': '2022-05-07', 'Habit': ['recFqsNfUNIe..."
1,2022-05-08,[recFqsNfUNIe88oRK],5,[recenm1UYUwAIcoya],recWwwlzwY1XqGQdE,2022-05-08T21:56:53.000Z,"{'Date': '2022-05-08', 'Habit': ['recFqsNfUNIe..."
2,2022-05-08,[rec24NGEkbvktkD9B],4,[recenm1UYUwAIcoya],recmIsNssDDyEl9io,2022-05-08T21:56:49.000Z,"{'Date': '2022-05-08', 'Habit': ['rec24NGEkbvk..."
3,2022-05-07,[rec24NGEkbvktkD9B],3,[recenm1UYUwAIcoya],recqlcZ0qzaBZ7EHW,2022-05-08T17:11:28.000Z,"{'Date': '2022-05-07', 'Habit': ['rec24NGEkbvk..."


In [45]:
df.index=pd.DatetimeIndex(df['Date'])
df=df.sort_index()
dfr=df.resample('D').sum() #Resample

In [46]:
dfr

,Score
Date,
2022-05-07,8
2022-05-08,9


In [31]:
goal_table=Table(api_key, base_id, 'Goals')


In [35]:
goal_table.get('recenm1UYUwAIcoya')['fields']['Goal']

'Personal Development'